In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [31]:
data = pd.read_csv('/content/drive/MyDrive/California_Fire_Incidents_2013-20.csv')

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1636 entries, 0 to 1635
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   AcresBurned          1633 non-null   float64
 1   Active               1636 non-null   bool   
 2   AdminUnit            1636 non-null   object 
 3   AirTankers           28 non-null     float64
 4   ArchiveYear          1636 non-null   int64  
 5   CalFireIncident      1636 non-null   bool   
 6   CanonicalUrl         1636 non-null   object 
 7   ConditionStatement   284 non-null    object 
 8   ControlStatement     105 non-null    object 
 9   Counties             1636 non-null   object 
 10  CountyIds            1636 non-null   object 
 11  CrewsInvolved        171 non-null    float64
 12  Dozers               123 non-null    float64
 13  Engines              191 non-null    float64
 14  Extinguished         1577 non-null   object 
 15  Fatalities           21 non-null     f

In [33]:
data.head()

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,...,Public,SearchDescription,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,...,True,The Rim Fire was east of Groveland along Highw...,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17T15:25:00Z,Finalized,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,...,True,The Powerhouse Fire burned in May and June 201...,"Powerhouse Fire, May 2013, June 2013, Angeles ...",2013-05-30T15:28:00Z,Finalized,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN
2,27531.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/7/15/mountain-fire/,NaN,NaN,Riverside,...,True,The Mountain Fire burned in July 2013 off High...,"Mountain Fire, July 2013, Highway 243, Highway...",2013-07-15T13:43:00Z,Finalized,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30T18:00:00Z,NaN
3,27440.0,False,Tahoe National Forest,NaN,2013,False,/incidents/2013/8/10/american-fire/,NaN,NaN,Placer,...,True,The American Fire burned in August 2013 off De...,"American Fire, August 2013, Deadwood Ridge, Fo...",2013-08-10T16:30:00Z,Finalized,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,2013-08-30T08:00:00Z,NaN
4,24251.0,False,Ventura County Fire/CAL FIRE,NaN,2013,True,/incidents/2013/5/2/springs-fire/,Acreage has been reduced based upon more accur...,NaN,Ventura,...,True,"The Springs Fire burned in May 2013, Southboun...","Springs Fire, May 2013, Highway 101, Camarillo...",2013-05-02T07:01:00Z,Finalized,6.0,10.0,46731fb8-3350-4920-bdf7-910ac0eb715c,2013-05-11T06:30:00Z,11.0


In [34]:
for col in data.columns:
    if data[col].dtype == 'object':
        data[col].fillna(data[col].mode()[0], inplace=True)
    else:
        data[col].fillna(data[col].median(), inplace=True)


<ipython-input-34-83397086bd0a>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].median(), inplace=True)
<ipython-input-34-83397086bd0a>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

In [35]:
#define fire risk as high if AcresBurnt > 500, otherwise low
data['FireRisk'] = np.where(data['AcresBurned'] > 500, 1, 0)

In [36]:
X = data[['Latitude', 'Longitude', 'CountyIds']]  # features
y = data['FireRisk']  # target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
#handling non-numeric columns with commas
for col in X_train.columns:
    # Remove commas if any and convert to float
    if X_train[col].dtype == 'object':
        X_train[col] = X_train[col].str.replace(',', '').astype(float)
        X_test[col] = X_test[col].str.replace(',', '').astype(float)


In [39]:
# Instantiate and train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [40]:
# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.7591463414634146
Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.89      0.85       256
           1       0.43      0.31      0.36        72

    accuracy                           0.76       328
   macro avg       0.63      0.60      0.60       328
weighted avg       0.73      0.76      0.74       328

Confusion Matrix:
 [[227  29]
 [ 50  22]]


In [41]:
# Get feature importances
importances = model.feature_importances_
feature_names = X.columns
feature_importances = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
print(feature_importances.sort_values(by='Importance', ascending=False))

     Feature  Importance
1  Longitude    0.396807
0   Latitude    0.391474
2  CountyIds    0.211719


In [79]:
def predict_fire_risk(model, data, county_id):
    # Filter the data for the specified city ID
    county_data = data[data['CountyIds'] == county_id]

    # Check if the city ID exists in the dataset
    if county_data.empty:
        print(f"No data available for city ID: {county_id}")
        return None

    # Select relevant features (same features used in training)
    features = ['Latitude', 'Longitude', 'CountyIds']
    county_data = county_data[features]

    # Predict the probability of each class (Low Risk and High Risk)
    probabilities = model.predict_proba(county_data)

    # Calculate the average probability for Low Risk and High Risk
    avg_low_risk_percentage = probabilities[:, 0].mean() * 100  # Average probability of Low Risk
    avg_high_risk_percentage = probabilities[:, 1].mean() * 100  # Average probability of High Risk

    # Display the overall percentage risk for the city
    print(f"County ID: {county_id}")
    print(f"Low Risk: {avg_low_risk_percentage:.2f}%")
    print(f"High Risk: {avg_high_risk_percentage:.2f}%")

    return avg_low_risk_percentage, avg_high_risk_percentage

# Print available city IDs for reference
print("Available county IDs:", data['CountyIds'].unique())

Available county IDs: ['55' '19' '33' '31' '56' '10' '47' '12' '52' '45' '37' '15,56' '49' '7'
 '4' '54' '42' '22' '33,' '27' '15' '9' '36' '32' '25' '40' '20' '14' '28'
 '35' '39' '17' '1' '11' '57' '9,34' '50' '48' '24' '23' '18' '3' '58'
 '29' '43' '5' '41' '30' '49,48,17,6' '28,17' '3,9' '53' ',40' '8' '17,28'
 '14,26' '26' '34' '2' '9,31' '51' '42,56' '28,49' '28,48' '16' '27,40'
 '46' '44' '6' '21' '6,11,17,23' '45,53' '19,56' '28,57' '6,17,23' '62'
 '61' '59' '62,37']


In [81]:
county_id = '59'
predictions = predict_fire_risk(model, data, county_id)

County ID: 59
Low Risk: 15.00%
High Risk: 85.00%
